In [1]:
import csv
import os
from collections import defaultdict
from datetime import datetime
from datetime import timedelta

import matplotlib
import numpy as np
import networkx as nx
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

In [2]:
total_colums = [
    # Ejemplar
    'TITULO', 'AUTOR', 'EDITORIAL', 'SUCURSAL', 'DESC_SUCURSAL', 'COD_BARRAS_LIBRO', 'NUM_TITULO', 'SIG_TOPOGRAFICA',
    'DEWEY', 'II_SUMARIO', 'I_SUMARIO', 'EDICION', 'LOCALIZACION_EJEMPLAR', 'DESC_LOCALIZACION_EJEMPLAR', 'POLITICA',
    'DESC_POLITICA_EJEMPLAR', 'ISBN', 'FECHA_REGISTRO_EJEMPLAR', 'FECHA_ADICION_EJEMPLAR', 'FECHA_PUB',
    'ESTADO_EJEMPLAR', 'PAIS_EJEMPLAR',
    # Usuario (I)
    'OCUPACION', 'NIVEL_EDUCACION', 'AREA_ESTUDIO',
    # Prestamo (I)
    'SUCURSAL_PRESTAMO', 'SUCURSAL_DEVOLUCION',
    # Usuario (II)
    'SUCURSAL_DEL_LECTOR', 'COD_BARRAS_LECTOR', 'CATEGORÍA',
    # Prestamo (II)
    'FECHA_PRESTAMO', 'FECHA_REAL_DEVOLUCION',
]

In [3]:
used_colums = [
    # Ejemplar
    'TITULO', 'AUTOR', 'SUCURSAL', 'COD_BARRAS_LIBRO', 'DEWEY', 'DESC_LOCALIZACION_EJEMPLAR',
    'FECHA_PUB','PAIS_EJEMPLAR',
    # Usuario (I)
    'OCUPACION','NIVEL_EDUCACION','AREA_ESTUDIO',
    # Prestamo (I)
    'SUCURSAL_PRESTAMO','SUCURSAL_DEVOLUCION',
    # Usuario (II)
    'SUCURSAL_DEL_LECTOR', 'COD_BARRAS_LECTOR', 'CATEGORÍA',
    # Prestamo (II)
    'FECHA_PRESTAMO','FECHA_REAL_DEVOLUCION'
]

# Datos limpios

In [4]:
%%time

dtypes = {
    'TITULO': str,
    'AUTOR': str,
    'SUCURSAL': str,
    'COD_BARRAS_LIBRO': str,
    'DEWEY': str,
    'DESC_LOCALIZACION_EJEMPLAR': str,
    'FECHA_PUB': str,
    'PAIS_EJEMPLAR': str,
    'OCUPACION': str,
    'NIVEL_EDUCACION': str,
    'AREA_ESTUDIO': str,
    'SUCURSAL_PRESTAMO': str,
    'SUCURSAL_DEVOLUCION': str,
    'SUCURSAL_DEL_LECTOR': str,
    'COD_BARRAS_LECTOR': str,
    'CATEGORÍA': str,
    'FECHA_PRESTAMO': str,
    'FECHA_REAL_DEVOLUCION': str,
    'DATE_PRESTAMO': pd.tslib.Timestamp,
    'DATE_DEVOLUCION': pd.tslib.Timestamp,
}

prestamos_df = pd.read_csv('prestamos_totales.csv', header=0, dtype=dtypes, parse_dates=['DATE_PRESTAMO', 'DATE_DEVOLUCION'])
prestamos_df = prestamos_df.fillna('')

CPU times: user 25.9 s, sys: 1.42 s, total: 27.3 s
Wall time: 27.4 s


In [5]:
prestamos_df.shape

(3054360, 20)

In [6]:
prestamos_df.head()

,TITULO,AUTOR,SUCURSAL,COD_BARRAS_LIBRO,DEWEY,DESC_LOCALIZACION_EJEMPLAR,FECHA_PUB,PAIS_EJEMPLAR,OCUPACION,NIVEL_EDUCACION,AREA_ESTUDIO,SUCURSAL_PRESTAMO,SUCURSAL_DEVOLUCION,SUCURSAL_DEL_LECTOR,COD_BARRAS_LECTOR,CATEGORÍA,FECHA_PRESTAMO,FECHA_REAL_DEVOLUCION,DATE_PRESTAMO,DATE_DEVOLUCION
0,Un bicho extraño,"Daporta, Mon.",ARM,2298061,863,Infantil,2010,sp,,,,ARM,ARM,ARM,88429527,CTA,22/01/2015,04/02/2015,2015-01-22,2015-02-04
1,Un bicho extraño,"Daporta, Mon.",ARM,2298061,863,Infantil,2010,sp,,,,ARM,ARM,ARM,88416381,CTA,31/03/2015,07/04/2015,2015-03-31,2015-04-07
2,Quisiera tener ...,"Zoboli, Giovanna.",ARM,2284458,853,Infantil,2010,mx,,,,ARM,ARM,ARM,88381353,CTA,07/11/2015,17/11/2015,2015-11-07,2015-11-17
3,Johannes Gutenberg,"Borràs Perelló, Lluís.",ARM,2288922,925,Infantil,2010,sp,Empleado,Técnico,Adm. de empresas,ARM,ARM,ARM,88007424,CEM,16/01/2015,16/01/2015,2015-01-16,2015-01-16
4,La fantástica leyenda de: la princesa y el dragón,"Alins, Sonia.",ARM,2288896,863,Infantil,2010,sp,,,,ARM,ARM,ARM,88429529,CTA,21/01/2015,28/01/2015,2015-01-21,2015-01-28


# Libros viajeros

In [7]:
sucursales = {
    'ARM': {'name': 'Armenia', 'latitude': 4.53, 'longitude': -75.68},
    'BAR': {'name': 'Barranquilla', 'latitude': 10.963889, 'longitude': -74.796389},
    'BLAA': {'name': 'Luis Angel Arango', 'latitude': 4.59677, 'longitude': -74.072848},
    'BUC': {'name': 'Bucaramanga', 'latitude': 7.133333, 'longitude': -73},
    'BUE': {'name': 'Buenaventura', 'latitude': 3.877222, 'longitude': -77.026667},
    'CAL': {'name': 'Cali', 'latitude': 3.420556, 'longitude': -76.522222},
    'CAR': {'name': 'Cartagena', 'latitude': 10.4, 'longitude': -75.5},
    'CASAGO': {'name': 'Casa Gómez Campuzano', 'latitude': 4.6628468, 'longitude': -74.0512116},
    'CATMAN': {'name': 'Catalogación y Mantenimiento de Colecciones', 'latitude': 4.5769749, 'longitude': -74.0880581},
    'CUC': {'name': 'Cúcuta', 'latitude': 7.894167, 'longitude': -72.503889},
    'ESPE': {'name': 'Especial material audiovisual', 'latitude': 4.59677, 'longitude': -74.072848},
    'FLO': {'name': 'Florencia', 'latitude': 1.614167, 'longitude': -75.611667},
    'GIR': {'name': 'Girardot', 'latitude': 4.305, 'longitude': -74.801667},
    'HON': {'name': 'Honda', 'latitude': 5.204167, 'longitude': -74.741667},
    'IBA': {'name': 'Ibagué', 'latitude': 4.433333, 'longitude': -75.233333},
    'IEC': {'name': 'Investigaciones Económicas', 'latitude': 4.5769749, 'longitude': -74.0880581},
    'IPI': {'name': 'Ipiales', 'latitude': 0.830278, 'longitude': -77.644444},
    'LET': {'name': 'Leticia', 'latitude': -4.205278, 'longitude': -69.932778},
    'MAN': {'name': 'Manizales', 'latitude': 5.1, 'longitude': -75.55},
    'MED': {'name': 'Medellín', 'latitude': 6.230833, 'longitude': -75.590556},
    'MON': {'name': 'Montería', 'latitude': 8.75, 'longitude': -75.883333},
    'NEI': {'name': 'Neiva', 'latitude': 2.998611, 'longitude': -75.304444},
    'PAS': {'name': 'Pasto', 'latitude': 1.207778, 'longitude': -77.277222},
    'PER': {'name': 'Pereira', 'latitude': 4.814278, 'longitude': -75.694558},
    'POP': {'name': 'Popayán', 'latitude': 2.454167, 'longitude': -76.609167},
    'QUI': {'name': 'Quibdó', 'latitude': 5.692222, 'longitude': -76.658056},
    'RIO': {'name': 'Riohacha', 'latitude': 11.544167, 'longitude': -72.906944},
    'SAN': {'name': 'San Andrés', 'latitude': 12.583333, 'longitude': -81.7},
    'SIN': {'name': 'Sincelejo', 'latitude': 9.295, 'longitude': -75.396111},
    'STM': {'name': 'Santa Marta', 'latitude': 11.241944, 'longitude': -74.205278},
    'TUN': {'name': 'Tunja', 'latitude': 5.533333, 'longitude': -73.366667},
    'VAL': {'name': 'Valledupar', 'latitude': 10.483333, 'longitude': -73.25},
    'VIL': {'name': 'Villavicencio', 'latitude': 4.15, 'longitude': -73.633333},
}

In [8]:
sucursales_bogota_no_blaa = ['CASAGO', 'IEC', 'ESPE', 'CATMAN']

In [9]:
sucursales_bogota = sucursales_bogota_no_blaa + ['BLAA']

## Modelos de viaje

In [10]:
# MODEL  SUC_LIBRO  SUC_PRéST  SUC_DEVOLUCIóN    %
#  m1       A          B          C             0.23%
#  m2       B          B          C             2.05%
#  m3       A          B          B            13.61%
#  m4       A          B          A             2.20%
#  m5       B          B          B            81.91%

#### Modelo 1

In [11]:
prestamos_m1_df = prestamos_df[
    (prestamos_df['SUCURSAL'] != prestamos_df['SUCURSAL_PRESTAMO']) &
    (prestamos_df['SUCURSAL_PRESTAMO'] != prestamos_df['SUCURSAL_DEVOLUCION']) &
    (prestamos_df['SUCURSAL_DEVOLUCION'] != prestamos_df['SUCURSAL'])
]

In [12]:
len(prestamos_m1_df)

7063

In [13]:
per = 100 * len(prestamos_m1_df) / len(prestamos_df)
'%.2f%%' % per

'0.23%'

#### Modelo 2

In [14]:
prestamos_m2_df = prestamos_df[
    (prestamos_df['SUCURSAL'] == prestamos_df['SUCURSAL_PRESTAMO']) &
    (prestamos_df['SUCURSAL_PRESTAMO'] != prestamos_df['SUCURSAL_DEVOLUCION']) &
    (prestamos_df['SUCURSAL_DEVOLUCION'] != prestamos_df['SUCURSAL'])
]

In [15]:
len(prestamos_m2_df)

62598

In [16]:
per = 100 * len(prestamos_m2_df) / len(prestamos_df)
'%.2f%%' % per

'2.05%'

#### Modelo 3

In [17]:
prestamos_m3_df = prestamos_df[
    (prestamos_df['SUCURSAL'] != prestamos_df['SUCURSAL_PRESTAMO']) &
    (prestamos_df['SUCURSAL_PRESTAMO'] == prestamos_df['SUCURSAL_DEVOLUCION']) &
    (prestamos_df['SUCURSAL_DEVOLUCION'] != prestamos_df['SUCURSAL'])
]

In [18]:
len(prestamos_m3_df)

415679

In [19]:
per = 100 * len(prestamos_m3_df) / len(prestamos_df)
'%.2f%%' % per

'13.61%'

#### Modelo 4

In [20]:
prestamos_m4_df = prestamos_df[
    (prestamos_df['SUCURSAL'] != prestamos_df['SUCURSAL_PRESTAMO']) &
    (prestamos_df['SUCURSAL_PRESTAMO'] != prestamos_df['SUCURSAL_DEVOLUCION']) &
    (prestamos_df['SUCURSAL_DEVOLUCION'] == prestamos_df['SUCURSAL'])
]

In [21]:
len(prestamos_m4_df)

67289

In [22]:
per = 100 * len(prestamos_m4_df) / len(prestamos_df)
'%.2f%%' % per

'2.20%'

#### Modelo 5

In [23]:
prestamos_m5_df = prestamos_df[
    (prestamos_df['SUCURSAL'] == prestamos_df['SUCURSAL_PRESTAMO']) &
    (prestamos_df['SUCURSAL_PRESTAMO'] == prestamos_df['SUCURSAL_DEVOLUCION']) &
    (prestamos_df['SUCURSAL_DEVOLUCION'] == prestamos_df['SUCURSAL'])
]

In [24]:
len(prestamos_m5_df)

2501731

In [25]:
per = 100 * len(prestamos_m5_df) / len(prestamos_df)
'%.2f%%' % per

'81.91%'

#### Total

In [26]:
len(prestamos_m1_df) + len(prestamos_m2_df) + len(prestamos_m3_df) + len(prestamos_m4_df) + len(prestamos_m5_df)

3054360

In [27]:
len(prestamos_df)

3054360

#### % préstamos sin viaje

In [28]:
per = 100 * len(prestamos_m5_df) / len(prestamos_df)
'%.2f%%' % per

'81.91%'

#### % préstamos con viaje

In [29]:
per = 100 * (len(prestamos_m1_df) + len(prestamos_m2_df) + len(prestamos_m3_df) + len(prestamos_m4_df)) / len(prestamos_df)
'%.2f%%' % per

'18.09%'

### Modelo 1

In [30]:
%%time

g1 = nx.MultiDiGraph()

for p in prestamos_m1_df.iterrows():
    s_libro = p[1]['SUCURSAL']
    s_prest = p[1]['SUCURSAL_PRESTAMO']
    s_devol = p[1]['SUCURSAL_DEVOLUCION']
    
    if not g1.has_node(s_libro):
        g1.add_node(
            s_libro,
            weight=0,
            nombre=sucursales[s_libro]['name'],
            latitude=sucursales[s_libro]['latitude'],
            longitude=sucursales[s_libro]['longitude']
        )
    g1.node[s_libro]['weight'] += 1
    if not g1.has_node(s_prest):
        g1.add_node(
            s_prest,
            weight=0,
            nombre=sucursales[s_prest]['name'],
            latitude=sucursales[s_prest]['latitude'],
            longitude=sucursales[s_prest]['longitude']
        )
    g1.node[s_prest]['weight'] += 1
    if not g1.has_node(s_devol):
        g1.add_node(
            s_devol,
            weight=0,
            nombre=sucursales[s_devol]['name'],
            latitude=sucursales[s_devol]['latitude'],
            longitude=sucursales[s_devol]['longitude']
        )
    g1.node[s_devol]['weight'] += 1
    
    if s_libro != s_prest:
        key = s_libro + '_' + s_prest + '_solicitud'
        if not g1.has_edge(s_libro, s_prest, key):
            g1.add_edge(s_libro, s_prest, key=key, label='solicitud', weight=0)
        g1[s_libro][s_prest][key]['weight'] += 1
    if s_prest != s_devol:
        key = s_prest + '_' + s_devol + '_viaje'
        if not g1.has_edge(s_prest, s_devol, key):
            g1.add_edge(s_prest, s_devol, key=key, label='viaje', weight=0)
        g1[s_prest][s_devol][key]['weight'] += 1
#     if s_devol != s_libro:
#         key = s_devol + '_' + s_libro + '_retorno'
#         if not graph.has_edge(s_devol, s_libro, key):
#             graph.add_edge(s_devol, s_libro, key=key, label='retorno', weight=0)
#         graph[s_devol][s_libro][key]['weight'] += 1

CPU times: user 1.9 s, sys: 0 ns, total: 1.9 s
Wall time: 1.9 s


In [31]:
g1.number_of_nodes()

32

In [32]:
g1.number_of_edges()

680

In [33]:
for s in sucursales_bogota_no_blaa:
    if s in g1:
        g1.remove_node(s)

In [34]:
g1.number_of_nodes()

29

In [35]:
g1.number_of_edges()

535

In [36]:
nx.write_gexf(g1, 'modelo1.gexf')

### Modelo 2

In [37]:
%%time

g2 = nx.MultiDiGraph()

for p in prestamos_df.iterrows():
    s_libro = p[1]['SUCURSAL']
    s_prest = p[1]['SUCURSAL_PRESTAMO']
    s_devol = p[1]['SUCURSAL_DEVOLUCION']
    
    if not g2.has_node(s_libro):
        g2.add_node(
            s_libro,
            weight=0,
            nombre=sucursales[s_libro]['name'],
            latitude=sucursales[s_libro]['latitude'],
            longitude=sucursales[s_libro]['longitude']
        )
    g2.node[s_libro]['weight'] += 1
    if not g2.has_node(s_prest):
        g2.add_node(
            s_prest,
            weight=0,
            nombre=sucursales[s_prest]['name'],
            latitude=sucursales[s_prest]['latitude'],
            longitude=sucursales[s_prest]['longitude']
        )
    g2.node[s_prest]['weight'] += 1
    if not g2.has_node(s_devol):
        g2.add_node(
            s_devol,
            weight=0,
            nombre=sucursales[s_devol]['name'],
            latitude=sucursales[s_devol]['latitude'],
            longitude=sucursales[s_devol]['longitude']
        )
    g2.node[s_devol]['weight'] += 1
    
    if s_libro != s_prest:
        key = s_libro + '_' + s_prest + '_solicitud'
        if not g2.has_edge(s_libro, s_prest, key):
            g2.add_edge(s_libro, s_prest, key=key, label='solicitud', weight=0)
        g2[s_libro][s_prest][key]['weight'] += 1
    if s_prest != s_devol:
        key = s_prest + '_' + s_devol + '_viaje'
        if not g2.has_edge(s_prest, s_devol, key):
            g2.add_edge(s_prest, s_devol, key=key, label='viaje', weight=0)
        g2[s_prest][s_devol][key]['weight'] += 1
#     if s_devol != s_libro:
#         key = s_devol + '_' + s_libro + '_retorno'
#         if not graph.has_edge(s_devol, s_libro, key):
#             graph.add_edge(s_devol, s_libro, key=key, label='retorno', weight=0)
#         graph[s_devol][s_libro][key]['weight'] += 1

CPU times: user 11min 14s, sys: 1.22 s, total: 11min 15s
Wall time: 11min 17s


In [38]:
g2.number_of_nodes()

33

In [39]:
g2.number_of_edges()

1388

In [40]:
for s in sucursales_bogota_no_blaa:
    if s in g2:
        g2.remove_node(s)

In [41]:
g2.number_of_nodes()

29

In [42]:
g2.number_of_edges()

1188

In [43]:
nx.write_gexf(g2, 'modelo2.gexf')

### Modelo 3

In [44]:
%%time

g3 = nx.MultiDiGraph()

for p in prestamos_df.iterrows():
    s_libro = p[1]['SUCURSAL']
    s_prest = p[1]['SUCURSAL_PRESTAMO']
    s_devol = p[1]['SUCURSAL_DEVOLUCION']
    
    if not g3.has_node(s_libro):
        g3.add_node(
            s_libro,
            weight=0,
            nombre=sucursales[s_libro]['name'],
            latitude=sucursales[s_libro]['latitude'],
            longitude=sucursales[s_libro]['longitude']
        )
    g3.node[s_libro]['weight'] += 1
    if not g3.has_node(s_prest):
        g3.add_node(
            s_prest,
            weight=0,
            nombre=sucursales[s_prest]['name'],
            latitude=sucursales[s_prest]['latitude'],
            longitude=sucursales[s_prest]['longitude']
        )
    g3.node[s_prest]['weight'] += 1
    if not g3.has_node(s_devol):
        g3.add_node(
            s_devol,
            weight=0,
            nombre=sucursales[s_devol]['name'],
            latitude=sucursales[s_devol]['latitude'],
            longitude=sucursales[s_devol]['longitude']
        )
    g3.node[s_devol]['weight'] += 1
    
    if s_libro != s_prest:
        key = s_libro + '_' + s_prest + '_solicitud'
        if not g3.has_edge(s_libro, s_prest, key):
            g3.add_edge(s_libro, s_prest, key=key, label='solicitud', weight=0)
        g3[s_libro][s_prest][key]['weight'] += 1
    if s_prest != s_devol:
        key = s_prest + '_' + s_devol + '_viaje'
        if not g3.has_edge(s_prest, s_devol, key):
            g3.add_edge(s_prest, s_devol, key=key, label='viaje', weight=0)
        g3[s_prest][s_devol][key]['weight'] += 1
#     if s_devol != s_libro:
#         key = s_devol + '_' + s_libro + '_retorno'
#         if not graph.has_edge(s_devol, s_libro, key):
#             graph.add_edge(s_devol, s_libro, key=key, label='retorno', weight=0)
#         graph[s_devol][s_libro][key]['weight'] += 1

CPU times: user 8min 27s, sys: 938 ms, total: 8min 27s
Wall time: 8min 28s


In [45]:
g3.number_of_nodes()

33

In [46]:
g3.number_of_edges()

1388

In [47]:
for s in sucursales_bogota_no_blaa:
    if s in g3:
        g3.remove_node(s)

In [48]:
g3.number_of_nodes()

29

In [49]:
g3.number_of_edges()

1188

In [50]:
nx.write_gexf(g3, 'modelo3.gexf')

### Modelo 4

In [51]:
%%time

g4 = nx.MultiDiGraph()

for p in prestamos_df.iterrows():
    s_libro = p[1]['SUCURSAL']
    s_prest = p[1]['SUCURSAL_PRESTAMO']
    s_devol = p[1]['SUCURSAL_DEVOLUCION']
    
    if not g4.has_node(s_libro):
        g4.add_node(
            s_libro,
            weight=0,
            nombre=sucursales[s_libro]['name'],
            latitude=sucursales[s_libro]['latitude'],
            longitude=sucursales[s_libro]['longitude']
        )
    g4.node[s_libro]['weight'] += 1
    if not g4.has_node(s_prest):
        g4.add_node(
            s_prest,
            weight=0,
            nombre=sucursales[s_prest]['name'],
            latitude=sucursales[s_prest]['latitude'],
            longitude=sucursales[s_prest]['longitude']
        )
    g4.node[s_prest]['weight'] += 1
    if not g4.has_node(s_devol):
        g4.add_node(
            s_devol,
            weight=0,
            nombre=sucursales[s_devol]['name'],
            latitude=sucursales[s_devol]['latitude'],
            longitude=sucursales[s_devol]['longitude']
        )
    g4.node[s_devol]['weight'] += 1
    
    if s_libro != s_prest:
        key = s_libro + '_' + s_prest + '_solicitud'
        if not g4.has_edge(s_libro, s_prest, key):
            g4.add_edge(s_libro, s_prest, key=key, label='solicitud', weight=0)
        g4[s_libro][s_prest][key]['weight'] += 1
    if s_prest != s_devol:
        key = s_prest + '_' + s_devol + '_viaje'
        if not g4.has_edge(s_prest, s_devol, key):
            g4.add_edge(s_prest, s_devol, key=key, label='viaje', weight=0)
        g4[s_prest][s_devol][key]['weight'] += 1
#     if s_devol != s_libro:
#         key = s_devol + '_' + s_libro + '_retorno'
#         if not graph.has_edge(s_devol, s_libro, key):
#             graph.add_edge(s_devol, s_libro, key=key, label='retorno', weight=0)
#         graph[s_devol][s_libro][key]['weight'] += 1

CPU times: user 8min 18s, sys: 717 ms, total: 8min 19s
Wall time: 8min 18s


In [52]:
g4.number_of_nodes()

33

In [53]:
g4.number_of_edges()

1388

In [54]:
for s in sucursales_bogota_no_blaa:
    if s in g4:
        g4.remove_node(s)

In [55]:
g4.number_of_nodes()

29

In [56]:
g4.number_of_edges()

1188

In [57]:
nx.write_gexf(g4, 'modelo4.gexf')